# Problem 1
## Formulate the Least Square Problem for Vapor-liquid equilibria data
<br>
<img src = "P1_question.png" alt = "MAE 598_HW2-P2a-page-001" title = "Nearest point page 1" />
<br>

In [102]:
import numpy as np
import torch as t
from torch.autograd import Variable
import matplotlib as plt

# Given data for a(1,2,3) for water(H2o) and 1,4 dioxane system(14_ds)

T = 20
a_H2o = t.tensor([8.07131, 1730.63, 233.426])
a_14_ds = t.tensor([7.43155, 1554.679, 240.337])

# Calculation of the saturation pressures by using Antoine equation

psat_H2o   = 10**(a_H2o[0]-(a_H2o[1]/(T+a_H2o[2])))
psat_14_ds = 10**(a_14_ds[0]-(a_14_ds[1]/(T+a_14_ds[2])))
psat       = t.tensor([psat_H2o,psat_14_ds],requires_grad=True, dtype =t.float64)


# Given measured data for x1,x2 and p
x1 = np.asarray(np.arange(0.0,1.1,0.1))
x2 = 1-np.asarray(x1)
x  = t.tensor([x1,x2], requires_grad=False, dtype =t.float64)
p  = t.tensor([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5],requires_grad=False, dtype =t.float64)

# Defining initial values for variables A1 and A2
A = Variable(t.tensor([1.0,1.0]),requires_grad=True)
# Definig model for vapor - liquid equilibria data
def model(x,A,psat):
    t_A12 = (A[0]*(A[1]*x[1]/(A[0]*x[0]+A[1]*x[1]))**2)
    t_A21 = (A[1]*(A[0]*x[0]/(A[0]*x[0]+A[1]*x[1]))**2)
    term1 = x[0]*t.exp(t_A12)*psat[0]
    term2 = x[1]*t.exp(t_A21)*psat[1]
    p_model = term1+term2
    return p_model

# Fix the step size
a = 0.001

for i in range(1000):  # TODO: change the termination criterion
    loss = t.sum((p-model(x,A,psat))**2)
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        A -= a * A.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        A.grad.zero_()
        
print("Vapor-liquid equilibria model:",p_model)
print("A12 and A21 values:",A.data.numpy())
print("Loss:",loss.data.numpy())


             

Vapor-liquid equilibria model: tensor([28.8241, 30.1302, 30.6279, 30.6336, 30.3132, 29.7235, 28.8288, 27.4982,
        25.4819, 22.3634, 17.4732], dtype=torch.float64,
       grad_fn=<AddBackward0>)
A12 and A21 values: [1.9584184 1.6891861]
Loss: 0.6702104913132515
